In [1]:
%classpath add mvn org.apache.spark spark-sql_2.11 2.1.0
org.apache.log4j.Logger.getRootLogger().setLevel(org.apache.log4j.Level.ERROR);


null

In [4]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._ 


import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._


In [5]:
val spark = SparkSession.builder() 
  .master("local[*]") 
  .config("spark.executor.memory", "3g")
  .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
  .appName("NohupReader")
  .getOrCreate()


org.apache.spark.sql.SparkSession@694b6248

In [21]:
import spark.implicits._

val research_home: String = scala.util.Properties.envOrElse("RESEARCH_HOME", "/home/acald013/Research/")
val folder = s"${research_home}Scripts/Python/"
val prefix = "nohup"

val nohup = spark.read.textFile(s"${folder}${prefix}.out")


org.apache.spark.sql.SparkSession$implicits$@7d0bb9f2

In [22]:
println(nohup.count)

8517


null

In [23]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Line(line: String, n: Long)
case class Run(runID: Long, date: String, method: String, cores: Int, epsilon: Double, mu: Int, delta: Int, methodTime: Double)
case class Stage(runID: Long, n: Long, timestamp: String, stage: String, stageTime: Double, load: Int, unit: String)
case class MDFrow(mdfID: Long, n: Long, method: String, epsilon: Double, mu: Int, delta: Int, stage: String, time: Double, load: Int, unit: String)

implicit class DatasetOps(ds: org.apache.spark.sql.Dataset[_]) {
    def display(rows: Int = 20) = {
        import com.twosigma.beakerx.scala.table.TableDisplay
        val columns = ds.columns
        val rowVals = ds.toDF.take(rows)
        val t = new TableDisplay(rowVals map (row => (columns zip row.toSeq).toMap))
        t.display()
    }
}

defined class Line
defined class Run
defined class Stage
defined class MDFrow
defined class DatasetOps


In [24]:
val lines = nohup.toDF("line").withColumn("n", monotonicallyIncreasingId).as[Line].cache()
val nLines = lines.count()
lines.show(10)

+--------------------+---+
|                line|  n|
+--------------------+---+
|FLOCKFINDER=Merge...|  0|
|WARNING:root:Sett...|  1|
|acald013@dblab-ra...|  2|
|acald013@dblab-ra...|  3|
|acald013@dblab-ra...|  4|
|acald013@dblab-ra...|  5|
|no org.apache.spa...|  6|
|starting org.apac...|  7|
|acald013@dblab-ra...|  8|
|acald013@dblab-ra...|  9|
+--------------------+---+
only showing top 10 rows



null

In [25]:
val indicesRun = lines.filter{ l => 
        l.line.contains("=== MergeLast Start ===") || l.line.contains("method=MergeLast,") ||
        l.line.contains("=== SpatialJoin Start ===") || l.line.contains("method=SpatialJoin,")
    }
    .orderBy("n")
    .select("n")
    .collect()
    .toList
    .map(_.getLong(0))
    .grouped(2)
    .toList
    .map(pair => (pair.head, pair.last))
    .filter(r => r._1 != r._2)
    .zipWithIndex
val indexRun = spark.createDataset(indicesRun)
    .flatMap{ pair => 
        (pair._1._1 to pair._1._2)
        .toList.map(v => (pair._2, v))
    }
    .toDF("runID","n")
    .cache
indexRun.show(10, truncate = false)

+-----+---+
|runID|n  |
+-----+---+
|0    |18 |
|0    |19 |
|0    |20 |
|0    |21 |
|0    |22 |
|0    |23 |
|0    |24 |
|0    |25 |
|0    |26 |
|0    |27 |
+-----+---+
only showing top 10 rows



null

In [27]:
val runs = indexRun.join(lines, "n").
    groupBy("runID").
    agg(max($"n").alias("n")).
    join(lines, "n").
    select("runID", "line").
    orderBy("runID").
    map{ row =>
        val runID = row.getInt(0)
        val line  = row.getString(1)
        var arr1  = line.split(" -> ")
        val date  = arr1(0)
        val arr2  = arr1(1).split(",")
        val method  = arr2(0).split("=")(1)
        val cores   = arr2(1).split("=")(1).toInt
        val epsilon = arr2(2).split("=")(1).toDouble
        val mu      = arr2(3).split("=")(1).toInt
        val delta   = arr2(4).split("=")(1).toInt
        val time    = arr2(5).split("=")(1).toDouble
        Run(runID, date, method, cores, epsilon, mu, delta, time)
    }.
    cache
val nRuns = runs.count()
runs.show(nRuns.toInt, truncate = false)

+-----+-----------------------+---------+-----+-------+---+-----+----------+
|runID|date                   |method   |cores|epsilon|mu |delta|methodTime|
+-----+-----------------------+---------+-----+-------+---+-----+----------+
|0    |2018-06-07 22:22:17,016|MergeLast|28   |100.0  |4  |5    |927.875   |
|1    |2018-06-08 10:40:22,043|MergeLast|28   |60.0   |4  |5    |654.964   |
|2    |2018-06-08 10:52:07,203|MergeLast|28   |70.0   |4  |5    |686.385   |
|3    |2018-06-08 11:03:42,503|MergeLast|28   |80.0   |4  |5    |677.862   |
|4    |2018-06-08 11:17:24,874|MergeLast|28   |90.0   |4  |5    |798.868   |
|5    |2018-06-08 14:33:29,207|MergeLast|28   |60.0   |4  |5    |631.36    |
|6    |2018-06-08 14:45:22,460|MergeLast|28   |70.0   |4  |5    |689.81    |
|7    |2018-06-08 14:56:50,832|MergeLast|28   |80.0   |4  |5    |670.759   |
|8    |2018-06-08 15:10:18,655|MergeLast|28   |90.0   |4  |5    |789.164   |
+-----+-----------------------+---------+-----+-------+---+-----+----------+

null

In [12]:
val sampleRuns = runs.filter($"method" === "MergeLast").filter($"mu" === 4 || $"mu" === 3).filter($"delta" === 3 || $"delta" === 5)
val nSampleRuns = sampleRuns.count()
sampleRuns.show(nSampleRuns.toInt, truncate = false)

+-----+-----------------------+---------+-----+-------+---+-----+----------+
|runID|date                   |method   |cores|epsilon|mu |delta|methodTime|
+-----+-----------------------+---------+-----+-------+---+-----+----------+
|0    |2018-06-07 22:22:17,016|MergeLast|28   |100.0  |4  |5    |927.875   |
|1    |2018-06-08 10:40:22,043|MergeLast|28   |60.0   |4  |5    |654.964   |
|2    |2018-06-08 10:52:07,203|MergeLast|28   |70.0   |4  |5    |686.385   |
|3    |2018-06-08 11:03:42,503|MergeLast|28   |80.0   |4  |5    |677.862   |
|4    |2018-06-08 11:17:24,874|MergeLast|28   |90.0   |4  |5    |798.868   |
+-----+-----------------------+---------+-----+-------+---+-----+----------+



null

In [15]:
val stages = lines.filter(_.line.contains("| ")).
    join(indexRun, "n").
    map{ m =>
        val n         = m.getLong(0)
        val line      = m.getString(1)
        val runID     = m.getInt(2)
        var arr1      = line.split(" -> ")
        val timestamp = arr1(0).trim
        val arr2      = arr1(1).split("\\|")
        val stage     = arr2(0).trim
        val time      = arr2(1).trim.dropRight(1).toDouble
        val arr3      = arr2(2).trim.split(" ")
        val load      = arr3(0).toInt
        val unit      = arr3(1)
        Stage(runID, n, timestamp, stage, time, load, unit)
    }.
    join(sampleRuns.select($"runID"), "runID").
    cache
    
println(stages.count())
stages.show(10, truncate = false)

355
+-----+---+-----------------------+----------------------------------+---------+-----+----------+
|runID|n  |timestamp              |stage                             |stageTime|load |unit      |
+-----+---+-----------------------+----------------------------------+---------+-----+----------+
|0    |19 |2018-06-07 22:06:55,075|Reporting locations at t=0...     |5.73     |18093|points    |
|0    |35 |2018-06-07 22:07:57,199|1.Set of disks for t_i...         |62.12    |3329 |disks     |
|0    |36 |2018-06-07 22:08:02,190|Reporting locations at t=4...     |4.99     |18548|points    |
|0    |52 |2018-06-07 22:09:05,539|2.Set of disks for t_i+delta...   |63.35    |3602 |disks     |
|0    |53 |2018-06-07 22:09:27,422|3.Joining timestams               |21.88    |220  |candidates|
|0    |54 |2018-06-07 22:09:39,911|Indexing candidates...            |1.07     |219  |candidates|
|0    |55 |2018-06-07 22:09:40,105|Getting expansions...             |0.19     |2    |expansions|
|0    |56 |2018-

null

In [16]:
val ml_stages = stages.join(runs, "runID").
    filter($"method" === "MergeLast").
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage".alias("stage0"), $"stageTime").
    withColumn("stage1", regexp_replace($"stage0", "Reporting locations at t=\\d+", "0.Reporting locations")).
    withColumn("stage2", regexp_replace($"stage1", "Checking internal timestamps", "4.Checking internals")).
    withColumn("stage3", regexp_replace($"stage2", "\\.\\.\\.", "")).
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage3".alias("stage"), $"stageTime").
    filter(!$"stage".rlike("4.Distance Join phase")).
    filter(!$"stage".rlike("5.Getting candidates"))
ml_stages.show(truncate = false)


+-----+---+---------+-------+---+-----+-------------------------------+---------+
|runID|n  |method   |epsilon|mu |delta|stage                          |stageTime|
+-----+---+---------+-------+---+-----+-------------------------------+---------+
|0    |19 |MergeLast|100.0  |4  |5    |0.Reporting locations          |5.73     |
|0    |35 |MergeLast|100.0  |4  |5    |1.Set of disks for t_i         |62.12    |
|0    |36 |MergeLast|100.0  |4  |5    |0.Reporting locations          |4.99     |
|0    |52 |MergeLast|100.0  |4  |5    |2.Set of disks for t_i+delta   |63.35    |
|0    |53 |MergeLast|100.0  |4  |5    |3.Joining timestams            |21.88    |
|0    |54 |MergeLast|100.0  |4  |5    |Indexing candidates            |1.07     |
|0    |55 |MergeLast|100.0  |4  |5    |Getting expansions             |0.19     |
|0    |56 |MergeLast|100.0  |4  |5    |Finding local maximals         |0.27     |
|0    |57 |MergeLast|100.0  |4  |5    |Prunning duplicates and subsets|2.46     |
|0    |164|Merge

null

In [17]:
val sj_stages = stages.join(runs, "runID").
    filter($"method" === "SpatialJoin").
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage".alias("stage0"), $"stageTime").
    withColumn("stage1", regexp_replace($"stage0", "\\.\\.\\.", "")).
    withColumn("stage2", regexp_replace($"stage1", "Reporting", "0.Reporting")).
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage2".alias("stage"), $"stageTime").
    filter(!$"stage".rlike("4.Distance Join phase")).
    filter(!$"stage".rlike("5.Getting candidates"))
sj_stages.show(truncate = false)



+-----+---+------+-------+---+-----+-----+---------+
|runID|n  |method|epsilon|mu |delta|stage|stageTime|
+-----+---+------+-------+---+-----+-----+---------+
+-----+---+------+-------+---+-----+-----+---------+



null

In [18]:
val data = runs.select($"runID", $"method", $"epsilon", $"mu", $"delta", $"methodTime".alias("time")).
    filter($"method" === "MergeLast").filter($"mu" === 4 || $"mu" === 3).filter($"delta" === 3 || $"delta" === 5).
    orderBy($"runID", $"epsilon", $"method").
    cache
println(data.count())
data.show(data.count().toInt, truncate = false)

5
+-----+---------+-------+---+-----+-------+
|runID|method   |epsilon|mu |delta|time   |
+-----+---------+-------+---+-----+-------+
|0    |MergeLast|100.0  |4  |5    |927.875|
|1    |MergeLast|60.0   |4  |5    |654.964|
|2    |MergeLast|70.0   |4  |5    |686.385|
|3    |MergeLast|80.0   |4  |5    |677.862|
|4    |MergeLast|90.0   |4  |5    |798.868|
+-----+---------+-------+---+-----+-------+



null

In [19]:
val d = data.collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}methods_${prefix}.csv" ))
pw.write(s"$d\n")
pw.close

null

In [20]:
val d = ml_stages.union(sj_stages).collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}stages_${prefix}.csv" ))
pw.write(s"$d\n")
pw.close

null

In [ ]:
val indicesMdf = lines.filter{ l => 
        l.line.contains(" -> Setting mu=") || l.line.contains(" ->   berlin0-10,") 
    }
    .orderBy("n")
    .select("n")
    .collect()
    .toList
    .map(_.getLong(0))
    .grouped(2)
    .toList
    .map(pair => (pair.head, pair.last))
    .filter(r => r._1 != r._2)
    .zipWithIndex
val indexMdf = spark.createDataset(indicesMdf)
    .flatMap{ pair => 
        (pair._1._1 to pair._1._2)
        .toList.map(v => (pair._2, v))
    }
    .toDF("mdfID","n")
    .cache
indexMdf.show(15, truncate = false)

In [ ]:
val mdfInfo = indexMdf.groupBy($"mdfID").agg(max($"n").alias("m")).orderBy($"m")
mdfInfo.show(10, truncate = false)

In [ ]:
val mdfInfo = indexMdf.groupBy($"mdfID").agg(max($"n").alias("n")).orderBy($"n").
    join(indexRun, "n").
    join(runs, "runID").
    join(lines, "n").
    select($"method", $"epsilon", $"mu", $"delta", $"line").
    withColumn("timestamp", substring($"line", 122, 124)).
    select($"method", $"epsilon", $"mu", $"delta", $"timestamp")

mdfInfo.show(truncate = false)

In [ ]:
val d = mdfInfo.collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}mdfInfo_${prefix}.csv" ))
pw.write(s"$d\n")
pw.close